In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /home/mohammad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import numpy
import random 
import tensorflow    as tf
import json 



In [3]:
with open("intents (2).json") as file:
    data = json.load(file)   


In [4]:
import numpy as np

# Define preprocessing functions
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

# Initialize lists for words, labels, training data, and output
words = []
labels = []
training_data = []
output_data = []

# Tokenize and stem each pattern, and prepare training data and output
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = tokenize(pattern)
        words.extend(tokens)
        training_data.append(tokens)
        output_data.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

# Stem and lowercase words, remove duplicates, and sort
words = sorted(list(set([stem(word) for word in words if word != "?"])))
labels = sorted(labels)

# Initialize empty output rows
out_empty = [0] * len(labels)

# Prepare training and output arrays
X_train = []
y_train = []

for idx, doc in enumerate(training_data):
    bag = []

    # Stem and lowercase words in the pattern
    tokens = [stem(word) for word in doc]

    # Create bag of words
    for word in words:
        bag.append(1) if word in tokens else bag.append(0)

    # Create output row
    output_row = list(out_empty)
    output_row[labels.index(output_data[idx])] = 1

    X_train.append(bag)
    y_train.append(output_row)

# Convert lists to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)


In [5]:
import tensorflow as tf
import numpy as np

# Define the neural network architecture
hidden1_size = 8
hidden2_size = 8

# Define the input and output shapes
input_shape = len(words)
output_shape = len(labels)

# Define the input layer
input_data = tf.keras.Input(shape=(input_shape,))

# Define the hidden layers
hidden1 = tf.keras.layers.Dense(hidden1_size, activation='relu')(input_data)
hidden2 = tf.keras.layers.Dense(hidden2_size, activation='relu')(hidden1)

# Define the output layer
output = tf.keras.layers.Dense(output_shape, activation='softmax')(hidden2)

# Create the model
model = tf.keras.Model(inputs=input_data, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Training loop
num_epochs = 100
batch_size = 8
for epoch in range(num_epochs):
    epoch_loss = 0
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        batch_loss = model.train_on_batch(batch_X, batch_y)
        epoch_loss += batch_loss[0]
    print('Epoch', epoch, 'completed out of', num_epochs, 'loss:', epoch_loss)

# Once trained, you can use the model for prediction
predictions = model.predict(X_train)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 301)]             0         
                                                                 
 dense (Dense)               (None, 8)                 2416      
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 99)                891       
                                                                 
Total params: 3379 (13.20 KB)
Trainable params: 3379 (13.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 0 completed out of 100 loss: 363.56739807128906
Epoch 1 completed out of 100 loss: 360.5812487602234
Epoch 2 completed out of 100 loss: 355.8002076148987

In [6]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [7]:
def chat():
    print("Start Talking with the bot(type quit to stop!)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        # Convert user input to bag of words
        bow = bag_of_words(inp, words)

        # Reshape the input data to match the model's input shape
        bow = bow.reshape(1, -1)  # Reshape to (1, 301)

        # Predict using the model
        results = model.predict(bow)[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        # Print response based on the predicted tag
        for intent in data["intents"]:
            if intent['tag'] == tag:
                print("Bot:", random.choice(intent['responses']))


In [8]:
chat() 



Start Talking with the bot(type quit to stop!)
1/1 [==============================] - 0s 12ms/step
Bot: Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.
1/1 [==============================] - 0s 13ms/step
Bot: 1)Provide clear fluids to sip, such as water, broth, or fruit juice diluted with water. 2)Serve bland foods, such as saltine crackers, plain bread, dry toast, rice, gelatin, or applesauce. 3)Avoid spicy or greasy foods and caffeinated or carbonated drinks until 48 hours after all symptoms have gone away. 
1/1 [==============================] - 0s 12ms/step
Bot: To treat a fever at home: 1)Drink plenty of fluids to stay hydrated. 2)Dress in lightweight clothing. 3)Use a light blanket if you feel chilled, until the chills end. 4)Take a